In [1]:
%load_ext autoreload
%autoreload 2

In [18]:
import typing as t
from fdua_competition.vectorstore import build_vectorstore
from fdua_competition.enums import Mode, VectorStoreOption
from fdua_competition.rag import ResearchAssistant, RAG
from fdua_competition.enums import ChatModelOption
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.vectorstores.base import VectorStore

In [14]:
vectorstore = build_vectorstore("one_assistant_test", Mode.TEST, VectorStoreOption.CHROMA)

[prepare_vectorstore] chroma: /fdua-competition/.fdua-competition/vectorstores/chroma
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/9.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/8.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/10.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/6.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/7.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/5.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/4.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-co

In [15]:
assistant = ResearchAssistant(vectorstore)

In [16]:
res = assistant.invoke("電通についてまとめて")
print(res)

query='電通についてまとめて' response='電通は1901年に創業し広告業界の発展に寄与してきた通信と広告の統合企業です。グローバル展開も行っています。' reason='電通の歴史や事業領域は創業からの情報や組織の成長が示されています。' organization_name='電通グループ' contexts=['日本の広告業を産業として躍進', '電通グループの根底に流れる価値観', '電通の発展に尽力する']


In [19]:
# RAGクラスを継承して、新しいRAGを作成する例

## レスポンスの型式を定義
class FactCheckerResponse(BaseModel):
    score: str = Field(description="score")
    reason: str = Field(description="reason")


# RAGクラスを継承して新しいRAGクラスを定義
# prompt_template(), build_payload(), output_structure() メソッドをオーバーライドする
class FactChecker(RAG):
    def __init__(
        self, vectorstore: VectorStore, chat_model_option: ChatModelOption = ChatModelOption.AZURE, language: str = "japanese"
    ):
        super().__init__(vectorstore=vectorstore, chat_model_option=chat_model_option)
        self.language = language

    @property
    def prompt_template(self) -> ChatPromptTemplate:
        """プロンプトのテンプレート"""
        return ChatPromptTemplate.from_messages(
            [
                ("system", "fact check given ai response"),
                ("system", "context:\n{context}"),
                ("user", "ai-response:\n{ai_response}"),
            ]
        )

    def build_payload(self, query: str) -> dict[str, t.Any]:
        """テンプレートこの辞書で埋める"""
        return {
            "ai_response": query,
            "language": self.language,
        }

    @property
    def output_structure(self) -> BaseModel:
        """出力型式を定義する"""
        return FactCheckerResponse


# [end: research_assistant]

In [22]:
fact_checker = FactChecker(vectorstore)
fact_checker.invoke(f"{res.query}: {res.response}")

FactCheckerResponse(score='1', reason='電通は1901年に創業し、通信と広告の統合企業として日本の広告業界の発展に寄与してきたのは正しい。特に、グローバル展開も含まれている点が非常に重要で、これも事実である。')